In [69]:
import numpy as np
from scipy.linalg import expm, eigh

# Step 1: Define the Quantum System (Hamiltonian H)
# -------------------------------------------------------------------------------
# a random symmetric (Hermitian) matrix.
n_size = 24
np.random.seed(42)
A = np.random.randn(n_size, n_size)
H = (A + A.T) / 2  
print("Hamiltonian H:\n", H)

Hamiltonian H:
 [[ 0.49671415 -0.34132351  0.49565341  0.74360191  0.03098345  0.2784475
   0.9195478   0.26102331 -0.12769032 -0.11513259 -0.62796922 -0.35914895
   0.26147707 -0.54393195 -1.27520751 -0.02147051 -0.88598189  0.21727148
  -0.58871545 -0.75966703  0.66886559 -0.43820943  0.16362535 -0.12848306]
 [-0.34132351  0.11092259 -1.45701687  0.97017084 -0.16979171 -0.6005406
   0.09005813  0.54927101 -0.629618   -0.64726477  0.35390424 -1.23431342
  -0.20691796 -0.57308024 -0.82478595  0.86480007  0.44443018  0.34290356
   0.30094699 -0.66817301 -1.21703122 -0.6034848  -0.6824777   0.65577153]
 [ 0.49565341 -1.45701687  0.32408397 -1.50241369 -0.33590427  1.0072353
  -0.10297559  0.02088284 -0.3330183  -0.39728796  0.41812536  1.30397822
  -0.34364824  0.55990992 -0.34670176 -0.65248339  0.5771409   1.03392745
   0.71517347  0.2249418  -0.62240515 -0.61875684  0.49999403  0.93781961]
 [ 0.74360191  0.97017084 -1.50241369  0.8219025  -0.07377003 -0.85042921
  -0.61434792 -1.40168

In [70]:
# Step 2: Choose an Initial Reference State
# -------------------------------------------------------------------------------
# Physically, this should overlap with Hartree-Fock.
phi0 = np.zeros(n_size)
phi0[0] = 1.0  # Start fully in the first basis state.
print("Initial state phi0:\n", phi0)

Initial state phi0:
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [71]:
############################################################
# Step 3: Evolve the State in Real Time
# -------------------------------------------------------------------------------
# Generate M time-evolved states: |n> = exp(-i H n dt) |phi0>
T = 10.0                    # Total time
M = 50                      # Time-steps in the simulation
delta_t = T/M               # Time-step size
states = []
for n in range(M):
    # Matrix exponential for time evolution
    U = expm(-1j * H * n * delta_t)
    states.append(U @ phi0)
states = np.array(states)  # shape (M, n_size): holds all |n> states

print(f"Generated {M} time-evolved states.")
print(np.shape(states))

Generated 50 time-evolved states.
(50, 24)


In [72]:
# ===============================================================================
# Step 4: Construct Filtered Fourier-Transformed States
# -------------------------------------------------------------------------------
# Choose J filter energies to target the spectrum region of interest.
J = 24
E_min, E_max = -7, 7
E_filters = np.linspace(E_min, E_max, J)
# Build filtered states: |ψ_j> = sum_n exp(i n E_j dt) |n>
psi = []
for E_j in E_filters:
    coeffs = np.exp(1j * np.arange(M) * E_j * delta_t)
    psi_j = np.sum(coeffs[:, None] * states, axis=0)
    psi.append(psi_j)
psi = np.array(psi)  # shape (J, n_size)
print(f"Constructed {J} filtered states.")
print(psi)

Constructed 24 filtered states.
[[ 7.24605092e-01-1.20409437e+00j -8.52639781e-02+3.91584154e-01j
  -3.93746665e-01+7.42836937e-01j -1.83494344e-01+1.04044968e-01j
  -9.49203554e-02+3.00767858e-01j  8.57661482e-02-4.80998758e-02j
   2.91099690e-02+3.37222416e-01j  1.72131507e-02-7.23192619e-02j
  -1.42959665e-01+4.38703504e-01j -1.66952651e-01+2.79555257e-01j
   4.81594205e-01-8.95602580e-01j  2.39986374e-01-4.79800083e-01j
  -2.78096177e-01+8.03125909e-01j -1.79700551e-01+8.83749804e-02j
   1.42902782e-01-6.72238779e-01j -1.18967974e-01+2.83551060e-01j
   9.46594641e-02-5.05025823e-01j  2.59485441e-01-5.77655659e-01j
   1.77334889e-01-7.68045287e-01j -2.10433567e-01+3.88228918e-01j
  -2.77462902e-01+2.21944284e-01j  9.19415269e-02-2.68062273e-01j
   3.33302766e-02-3.04014037e-01j  4.51675181e-01-1.12600176e+00j]
 [ 5.44025238e-01-1.14886891e+00j  1.94459639e-01-4.29777230e-01j
  -4.40520343e-02-1.61750623e-01j -1.95891542e-01+1.11024344e-01j
   2.23503806e-02+1.53760530e-01j  4.243012

In [73]:
# ===============================================================================
# Step 5: Compute Overlap and Hamiltonian Matrices
# -------------------------------------------------------------------------------
# S[j,j']: Overlap matrix in the filter basis
# F[j,j']: Hamiltonian in the filter basis
S = np.zeros((J, J), dtype=complex)
F = np.zeros((J, J), dtype=complex)
for j in range(J):
    for jp in range(J):
        S[j, jp] = np.vdot(psi[j], psi[jp])         # Eq: S_{jj'} = <ψ_j | ψ_{j'}>
        F[j, jp] = np.vdot(psi[j], H @ psi[jp])     # Eq: F_{jj'} = <ψ_j | H | ψ_{j'}>

print("Overlap matrix S:\n", S)
print("Hamiltonian matrix F:\n", F)

Overlap matrix S:
 [[ 9.33952577e+00+0.00000000e+00j -1.31919290e+00+2.11515600e-01j
   2.02263549e+00-6.65721495e-01j  3.92579357e+00-2.02862515e+00j
   2.91196785e+00-2.14974781e+00j  8.31464435e+00-8.47450922e+00j
  -8.66663452e+00+1.22198108e+01j -3.64852415e+00+7.40295928e+00j
  -5.61223131e-01+1.82121636e+00j -4.94723037e-01+3.51212578e+00j
   1.60780014e-02+8.44186555e-01j  1.26513427e+00+7.03117434e+00j
   4.54717400e-01+1.29779245e+00j  5.59520205e+00+1.03401596e+01j
   3.56827564e+00+4.64564259e+00j  1.08119243e+00+1.02112369e+00j
  -1.69379886e+00-1.15345020e+00j -2.06898719e+00-9.71172579e-01j
   5.91296179e+00+1.69953857e+00j -7.42282624e-01-9.01798693e-02j
  -1.03897098e+00+3.95899183e-02j -1.07884857e+00+2.15404978e-01j
  -7.78022406e-01+2.89350363e-01j -1.32851315e+00+7.51928168e-01j]
 [-1.31919290e+00-2.11515600e-01j  6.73648483e+00-1.38777878e-17j
   5.70796962e+00-9.15199456e-01j  7.09852819e+00-2.33637886e+00j
   4.93300999e+00-2.54909687e+00j  1.24143115e+01-9.1648

In [74]:
# ===============================================================================
# Step 6: Solve the Generalized Eigenvalue Problem
# -------------------------------------------------------------------------------
# Finds E and c solving F c = E S c (standard in filter diagonalization)
# Eigenvalues E approximate eigenenergies of H near our chosen filter energies.
Eigvals, Eigvecs = eigh(F, S)
print("Computed Eigenvalues:\n", Eigvals)


Computed Eigenvalues:
 [-6.75956315 -4.80137205 -4.41267902 -4.16970497 -3.58792366 -3.17561363
 -2.87360608 -2.20427646 -1.74715978 -1.37078227 -1.0764462  -0.6869819
 -0.35845538  0.45915719  0.94152617  1.58575779  1.81856705  2.22254545
  3.03980074  3.88454002  4.22134803  4.92630963  5.80436737  6.53460187]


In [75]:
# ===============================================================================
# Step 7 (Optional): Tune Parameters
# -------------------------------------------------------------------------------
# For larger or ill-conditioned problems, tune M, J, [E_min,E_max], or regularize S.
# (Can add: S += np.eye(J) * 1e-6 if S is near-singular)


In [76]:
# # Regularize if S is not positive definite
# epsilon = 1e-6
# S_reg = S + np.eye(J) * epsilon

# # Solve generalized eigenproblem
# Eigvals, Eigvecs = eigh(F, S_reg)


In [77]:
# ===============================================================================
# Step 8: Output Results
# -------------------------------------------------------------------------------
print("Approximated eigenvalues near selected filters:", Eigvals.real)
# Eigenvalues should be checked against the full matrix diagonalization.
# accurate Eigvals = eigh(H)[0]
accurate_Eigvals = eigh(H)[0]
print("Accurate eigenvalues from full diagonalization:", accurate_Eigvals)

Approximated eigenvalues near selected filters: [-6.75956315 -4.80137205 -4.41267902 -4.16970497 -3.58792366 -3.17561363
 -2.87360608 -2.20427646 -1.74715978 -1.37078227 -1.0764462  -0.6869819
 -0.35845538  0.45915719  0.94152617  1.58575779  1.81856705  2.22254545
  3.03980074  3.88454002  4.22134803  4.92630963  5.80436737  6.53460187]
Accurate eigenvalues from full diagonalization: [-6.75956315 -4.80137205 -4.41267902 -4.16970497 -3.58792366 -3.17561363
 -2.87360608 -2.20427646 -1.74715978 -1.37078227 -1.07644621 -0.6869819
 -0.35845538  0.45915719  0.94152617  1.58575779  1.81856705  2.22254545
  3.03980074  3.88454002  4.22134803  4.92630963  5.80436737  6.53460187]


In [78]:
import numpy as np
from scipy.linalg import eigh, eig

def krylov_basis_matrices(H, psi0, times, fH=lambda x: x):
    """
    Construct subspace matrices for Krylov diagonalization.

    Args:
    - H: Hamiltonian matrix (NxN)
    - psi0: initial state vector (N)
    - times: array of M time steps [t0, t1, ..., t_{M-1}]
    - fH: function of H to define F matrix (H by default)

    Returns:
    - F: (M x M) matrix with elements <n|f(H)|m>
    - S: (M x M) overlap matrix <n|m>
    """
    M = len(times)
    N = H.shape[0]
    # Precompute evolution states |n> = exp(i * t_n * H) |psi0>
    basis = np.zeros((M, N), dtype=complex)
    for n, t in enumerate(times):
        U = scipy.linalg.expm(1j * t * H)  # time-evolution operator
        basis[n] = U @ psi0

    # Construct overlap matrix S_{nm} = <n|m>
    S = np.zeros((M, M), dtype=complex)
    # Construct F_{nm} = <n| f(H) |m>
    F = np.zeros((M, M), dtype=complex)
    for n in range(M):
        for m in range(M):
            S[n, m] = np.vdot(basis[n], basis[m])
            F[n, m] = np.vdot(basis[n], fH(H) @ basis[m])

    return F, S


In [79]:
def filter_basis_matrices(H, psi0, times, filter_energies, fH=lambda x: x):
    """
    Construct subspace matrices for Filter Diagonalization.

    Args:
    - H: Hamiltonian matrix (NxN)
    - psi0: initial state vector (N)
    - times: array of time points [0, ..., M-1]
    - filter_energies: array of filter energies E_j, length J
    - fH: function of H

    Returns:
    - F: (J x J) matrix
    - S: (J x J) overlap matrix
    """
    M = len(times)
    J = len(filter_energies)
    N = H.shape[0]

    # Precompute time-evolved states
    basis_time = np.zeros((M, N), dtype=complex)
    for n, t in enumerate(times):
        U = scipy.linalg.expm(1j * t * H)
        basis_time[n] = U @ psi0

    # Construct filter basis states |j> = sum_n e^{i n(E_j)} |n>
    basis_filter = np.zeros((J, N), dtype=complex)
    for j, Ej in enumerate(filter_energies):
        for n, t in enumerate(times):
            phase = np.exp(-1j * t * Ej)  # filter phase factor
            basis_filter[j] += phase * basis_time[n]

    # Compute overlap S_jk = <j|k>
    S = np.zeros((J, J), dtype=complex)
    F = np.zeros((J, J), dtype=complex)
    for j in range(J):
        for k in range(J):
            S[j, k] = np.vdot(basis_filter[j], basis_filter[k])
            F[j, k] = np.vdot(basis_filter[j], fH(H) @ basis_filter[k])

    return F, S



In [80]:
def transformation_matrix(times, filter_energies):
    """
    Construct transformation matrix W relating Krylov and Filter bases.

    Rows indexed by time steps n
    Columns indexed by filter energies j

    W_{n,j} = exp(i * t_n * E_j)
    """
    M = len(times)
    J = len(filter_energies)
    W = np.zeros((M, J), dtype=complex)
    for n, t in enumerate(times):
        for j, Ej in enumerate(filter_energies):
            W[n,j] = np.exp(1j * t * Ej)
    return W


In [81]:
def solve_generalized_eigenproblem(F, S):
    """
    Solve generalized eigenvalue problem Fc = lambda S c.

    Returns eigenvalues and eigenvectors.
    """
    eigvals, eigvecs = eigh(F, S)
    return eigvals, eigvecs



In [82]:
# Example usage: given Hamiltonian H, initial state psi0, times and filter_energies
# Fk, Sk = krylov_basis_matrices(H, psi0, times, fH=lambda H: H)
# Fj, Sj = filter_basis_matrices(H, psi0, times, filter_energies, fH=lambda H: H)
# W = transformation_matrix(times, filter_energies)
# # Relation: W Fk W^† = Fj, W Sk W^† = Sj

In [83]:
# Step 1: Define the Quantum System (Hamiltonian H)
# -------------------------------------------------------------------------------
# a random symmetric (Hermitian) matrix.
n_size=24
np.random.seed(42)
A = np.random.randn(n_size, n_size)
H = (A + A.T) / 2  
print("Hamiltonian H:\n", H)

Hamiltonian H:
 [[ 0.49671415 -0.34132351  0.49565341  0.74360191  0.03098345  0.2784475
   0.9195478   0.26102331 -0.12769032 -0.11513259 -0.62796922 -0.35914895
   0.26147707 -0.54393195 -1.27520751 -0.02147051 -0.88598189  0.21727148
  -0.58871545 -0.75966703  0.66886559 -0.43820943  0.16362535 -0.12848306]
 [-0.34132351  0.11092259 -1.45701687  0.97017084 -0.16979171 -0.6005406
   0.09005813  0.54927101 -0.629618   -0.64726477  0.35390424 -1.23431342
  -0.20691796 -0.57308024 -0.82478595  0.86480007  0.44443018  0.34290356
   0.30094699 -0.66817301 -1.21703122 -0.6034848  -0.6824777   0.65577153]
 [ 0.49565341 -1.45701687  0.32408397 -1.50241369 -0.33590427  1.0072353
  -0.10297559  0.02088284 -0.3330183  -0.39728796  0.41812536  1.30397822
  -0.34364824  0.55990992 -0.34670176 -0.65248339  0.5771409   1.03392745
   0.71517347  0.2249418  -0.62240515 -0.61875684  0.49999403  0.93781961]
 [ 0.74360191  0.97017084 -1.50241369  0.8219025  -0.07377003 -0.85042921
  -0.61434792 -1.40168

In [84]:
import scipy
psi0 = np.zeros(n_size)
psi0[0] = 1.0  # Start fully in the first basis state.
print("Initial state phi0:\n", psi0)
times=np.linspace(0, 10, 50)  # M=25 time steps from t=0 to t=10

Initial state phi0:
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [85]:
Fk, Sk = krylov_basis_matrices(H, psi0, times, fH=lambda H: H)
print("Krylov F matrix:\n", Fk)
print("Krylov S matrix:\n", Sk)

Krylov F matrix:
 [[ 0.49671415+0.00000000e+00j  0.41623718+1.30949620e+00j
   0.24766058+1.88540974e+00j ...  0.78004763+4.75396956e-04j
   0.76225906-1.54804149e-01j  0.10746842-3.06449141e-01j]
 [ 0.41623718-1.30949620e+00j  0.49671415-1.66533454e-16j
   0.41623718+1.30949620e+00j ...  0.13718085+1.76198556e-01j
   0.78004763+4.75396956e-04j  0.76225906-1.54804149e-01j]
 [ 0.24766058-1.88540974e+00j  0.41623718-1.30949620e+00j
   0.49671415+3.33066907e-16j ... -0.69727972+4.05534208e-01j
   0.13718085+1.76198556e-01j  0.78004763+4.75396956e-04j]
 ...
 [ 0.78004763-4.75396956e-04j  0.13718085-1.76198556e-01j
  -0.69727972-4.05534208e-01j ...  0.49671415-5.55111512e-17j
   0.41623718+1.30949620e+00j  0.24766058+1.88540974e+00j]
 [ 0.76225906+1.54804149e-01j  0.78004763-4.75396956e-04j
   0.13718085-1.76198556e-01j ...  0.41623718-1.30949620e+00j
   0.49671415+0.00000000e+00j  0.41623718+1.30949620e+00j]
 [ 0.10746842+3.06449141e-01j  0.76225906+1.54804149e-01j
   0.78004763-4.75396956

In [86]:
filter_energies = np.linspace(-7, 7, 24)  # J=24 filter energies from -7 to 7
Fj, Sj = filter_basis_matrices(H, psi0, times, filter_energies, fH=lambda H: H)
print("Filter F matrix:\n", Fj)
print("Filter S matrix:\n", Sj)

Filter F matrix:
 [[-4.94151530e+01+1.77635684e-15j  2.20780683e+01+2.17315399e+00j
   1.79865551e+00+3.57549040e-01j -8.20854019e+00-2.48840988e+00j
  -3.45399143e+00-1.42971289e+00j -2.89656399e+01-1.54712162e+01j
   5.12672241e+01+3.42288062e+01j  2.41027102e+01+1.97635411e+01j
   9.77458246e+00+9.76514435e+00j  8.04611644e+00+9.79336447e+00j
   6.42104614e+00+9.59722557e+00j  5.19543547e+00+9.70446729e+00j
   3.35657426e+00+8.08690973e+00j  1.52536144e+00+5.01427105e+00j
   2.12842560e+00+1.06532475e+01j -8.86621489e-03-8.91920063e-02j
   4.62526168e-03+4.78783961e+00j -6.86120602e-01+7.04037440e+00j
   4.42044109e+00-2.23499759e+01j  1.19382371e-01-3.95182364e-01j
   7.81151726e-02-1.89232706e-01j -4.33284940e-01+8.13097260e-01j
  -6.72370296e-01+1.00917257e+00j -1.00587238e+00+1.22913573e+00j]
 [ 2.20780683e+01-2.17315399e+00j -2.77696952e+01+2.22044605e-16j
  -1.88146516e+01-1.85193444e+00j -2.37544033e+01-4.72206270e+00j
  -1.22959750e+01-3.72751121e+00j -4.29459299e+01-1.77766

In [87]:
#solve generalized eigenproblems    
eigvals_j, eigvecs_j = solve_generalized_eigenproblem(Fj, Sj)
print("Filter Eigenvalues:\n", eigvals_j)
print("Accurate eigenvalues from full diagonalization:", eigh(H)[0])

Filter Eigenvalues:
 [-6.75956315 -4.80137205 -4.41267902 -4.16970497 -3.58792366 -3.17561363
 -2.87360608 -2.20427646 -1.74715978 -1.37078227 -1.07644621 -0.6869819
 -0.35845538  0.45915719  0.94152617  1.58575779  1.81856705  2.22254545
  3.03980074  3.88454002  4.22134803  4.92630963  5.80436737  6.53460187]
Accurate eigenvalues from full diagonalization: [-6.75956315 -4.80137205 -4.41267902 -4.16970497 -3.58792366 -3.17561363
 -2.87360608 -2.20427646 -1.74715978 -1.37078227 -1.07644621 -0.6869819
 -0.35845538  0.45915719  0.94152617  1.58575779  1.81856705  2.22254545
  3.03980074  3.88454002  4.22134803  4.92630963  5.80436737  6.53460187]


In [88]:
# Regularize S if needed
eps = 1e-8
Sk_reg = Sk + eps * np.eye(Sk.shape[0])
eigvals_k, eigvecs_k = eigh(Fk, Sk_reg)
print("Krylov Eigenvalues:\n", eigvals_k)

Krylov Eigenvalues:
 [-6.75956275e+00 -4.80135949e+00 -4.41174243e+00 -4.16396349e+00
 -3.58785002e+00 -3.14091323e+00 -2.87253028e+00 -2.18955799e+00
 -1.74222479e+00 -1.36577393e+00 -1.00649181e+00 -6.70767197e-01
 -3.58388926e-01 -1.73542109e-07 -1.52970840e-07 -1.39502914e-07
 -1.17020641e-07 -9.80248528e-08 -7.95620019e-08 -7.64458808e-08
 -6.67085758e-08 -5.45924811e-08 -4.41205870e-08 -3.51509044e-08
 -1.80283204e-08 -7.39741952e-09  8.53066006e-09  1.53757959e-08
  3.55308782e-08  4.53517389e-08  5.63561049e-08  7.83513859e-08
  8.15309336e-08  9.95436710e-08  1.11698184e-07  1.29928093e-07
  1.46944789e-07  1.72413299e-07  1.93598551e-07  4.58980455e-01
  9.41522740e-01  1.58574232e+00  1.81854883e+00  2.22254267e+00
  3.03979613e+00  3.88453997e+00  4.22134756e+00  4.92630930e+00
  5.80436515e+00  6.53460173e+00]


In [89]:
# Remove linearly dependent components in Sk
w, v = np.linalg.eigh(Sk)
thresh = 1e-6
mask = w > thresh
# Project Fk and Sk into well-conditioned subspace
Fk_good = v[:,mask].conj().T @ Fk @ v[:,mask]
Sk_good = v[:,mask].conj().T @ Sk @ v[:,mask]
eigvals_k, eigvecs_k = eigh(Fk_good, Sk_good)
print("Krylov Eigenvalues (well-conditioned):\n", eigvals_k)
# get the timesteps that has mask
times_good = times[mask] 

Krylov Eigenvalues (well-conditioned):
 [-6.75956281 -4.80131582 -4.40828838 -4.13987771 -3.58759168 -2.98215998
 -2.85931352 -2.11426003 -1.71908407 -1.35228727 -0.76868065 -0.35941112
  0.45407342  0.94146318  1.58553708  1.81832354  2.22251188  3.0397586
  3.8845397   4.22134523  4.92630886  5.80436622  6.53460183]


In [90]:
W = transformation_matrix(times_good, filter_energies)
print("Transformation matrix W:\n", W)

Transformation matrix W:
 [[ 0.64305408-0.76582077j -0.79007063+0.61301582j  0.90156307-0.4326477j
  -0.97251833+0.23282634j  0.99974602-0.02253637j -0.98202192-0.18876692j
   0.92014295+0.39158263j -0.81689138-0.57679153j  0.67690975+0.73606603j
  -0.50649207-0.86224462j  0.31330087+0.94965392j -0.10602267-0.99436371j
  -0.10602267+0.99436371j  0.31330087-0.94965392j -0.50649207+0.86224462j
   0.67690975-0.73606603j -0.81689138+0.57679153j  0.92014295-0.39158263j
  -0.98202192+0.18876692j  0.99974602+0.02253637j -0.97251833-0.23282634j
   0.90156307+0.4326477j  -0.79007063-0.61301582j  0.64305408+0.76582077j]
 [-0.66693806-0.74511316j  0.38335271+0.92360202j -0.05672493-0.99838984j
  -0.27627186+0.96107953j  0.57824915-0.81586024j -0.81530124+0.57903703j
   0.96081219-0.27720018j -0.99844418-0.05576042j  0.92397193+0.38246029j
  -0.7457571 -0.66621794j  0.48380947+0.87517335j -0.16754024-0.98586524j
  -0.16754024+0.98586524j  0.48380947-0.87517335j -0.7457571 +0.66621794j
   0.9239719

In [91]:
def check_assertions(Fk, Sk, Fj, Sj, W, tol=1e-10):
    # Compute the transformed matrices
    print("shape of W:", W.shape)   
    print("shape of Fk:", Fk.shape)
    print("shape of Sk:", Sk.shape)
    print("shape of Fj:", Fj.shape)
    print("shape of Sj:", Sj.shape)
    W_dag = W.conj().T
    lhs_F = W_dag @ Fk @ W
    lhs_S = W_dag @ Sk @ W
    print(scipy.linalg.norm(lhs_F - Fj))
    print(scipy.linalg.norm(lhs_S - Sj))
    # Check if they are close to Fj and Sj respectively
    F_equal = np.allclose(lhs_F, Fj, atol=tol)
    S_equal = np.allclose(lhs_S, Sj, atol=tol)
    
    assert F_equal, "Assertion failed: W† Fk W != Fj"
    assert S_equal, "Assertion failed: W† Sk W != Sj"
    print("Assertions passed: W† Fk W = Fj and W† Sk W = Sj")

In [92]:
check_assertions(Fk_good, Sk_good, Fj, Sj, W,tol =1e-10)

shape of W: (23, 24)
shape of Fk: (23, 23)
shape of Sk: (23, 23)
shape of Fj: (24, 24)
shape of Sj: (24, 24)
2528.975545152526
970.9649539693553


AssertionError: Assertion failed: W† Fk W != Fj